In [3]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import re
import requests
import pickle

In [3]:
BASE = 'https://www.mass.gov'
start = 'https://www.mass.gov/guides/list-of-court-forms-by-topic'
connection = requests.get(start)
html = connection.text
soup = bs4.BeautifulSoup(html,"lxml")
# print(soup)
connection.close()
div = soup.findAll("div", class_ = "ma__rich-text")

In [ ]:
hreflist = []
for category in div:
    type_of_form = category.ul
    if type_of_form!=None:
        for a in type_of_form.findAll("a"):
            hreflist.append(a['href'])
            
                        
print(hreflist)

In [5]:
formcategory = BASE + hreflist[0]
# print(formcategory)
connection = requests.get(formcategory)
html = connection.text
soup = bs4.BeautifulSoup(html,"lxml")
print(formcategory)
connection.close()


https://www.mass.gov/lists/probate-and-family-court-forms-for-wills-estates-and-trusts#accounting-forms-


In [2]:
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import date
import os
from os import walk
import os.path
from os import path
import uuid

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()

    import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup

In [7]:
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)
    

In [ ]:
errors = {}
pdfs = {}
redirections = []
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","status"])
form_dest = "PDFS/D/"
today = date.today().strftime("%Y-%m-%d")
#remove the parenthesis
pattern = r"\([^()]*\)"
source = "https://www.mass.gov/guides/list-of-court-forms-by-topic"
jur = "MA"
status = 0
for href in range(len(hreflist)):
    time.sleep(2)
    formcategory = BASE + hreflist[href]
    connection = requests.get(formcategory)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    print(formcategory)
    connection.close()
    try:
        groups = soup.find("div",class_ = "ma__stacked-row stacked-row-column")
        section = groups.find_all("h2")
        links = soup.find_all("div", class_ = "ma__form-downloads__links")
    except:
        continue
    
    for group in range(len(section)-1): #-1 because last is contact
            forms = links[group].find_all(class_ = "ma__download-link__file-link")
            redirects = links[group].find_all(class_ = "js-clickable-link")
            curr_group = section[group].text
            print(curr_group)
            
            # redirects result into an error download
            if redirects!=None:
                for i in redirects:
                    if "https" not in i['href']:
                        inner_url = BASE + i['href']
                        time.sleep(2)
                        inner_soup = connect_to_page(inner_url)
                        inner_div = inner_soup.findAll("a", href= True)
                        for i in inner_div:
                            if "courtforms" in i['href']:
                                url = i['href']
                                title = re.sub(pattern,"",i.text)
                                filename = url.split("//")[1].split("/")[-1] +".pdf"
                                
                                if filename not in pdfs:
                                    print(title)
                                    print(filename)
                                    pdfs[filename] = title
                                    fileid = hashme(url)
                                    status = download_pdf(url,form_dest,fileid +".pdf")
                                    
                                    if status=='error':
                                        print(status, url)
                                        status = 1
                                        errors[filename] = title
                                    else: status = 0
                                    files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,curr_group,url,filename,today,status]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","status"]))
                                status = 0
                    else:
                        redirections.append(i['href'])


            for f in forms:
                url = f['href']
                filename = url.split("//")[1].split("/")[-2] +".pdf"
                
                if filename not in pdfs:
                    title = re.sub(pattern,"",f.text.split(",")[2].split("\n")[1].lstrip())
                    pdfs[filename] = title
                    fileid = hashme(url)
                    #print(curr_group)
                    print(title)
                    print(filename)
                    status = download_pdf(url,form_dest,fileid+".pdf")
                    
                    if status=='error':
                        print(status, url)
                        status = 1
                        errors[filename] = title
                    else: status = 0
                    files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,curr_group,url,filename,today,status]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","status"]))
                status = 0
if len(files_df) > 0:
    files_df.to_csv("All.csv",index=False, encoding="utf-8")

# with open('C_errors.pickle', 'wb') as error_file:
#     pickle.dump(errors, error_file)
# with open('C_pdfs.pickle', 'wb') as pdfs_file:
#     pickle.dump(pdfs, pdfs_file)
# with open('C_redirections.pickle', 'wb') as inner_sites_file:
#     pickle.dump(redirections, inner_sites_file)

The code below was written deal with multiple downloading errors that occured. Instead of directly downloading the forms from the url, I used selenium to click a download button.

In [9]:
import urllib.request
def download(download_url, filename):
    response = urllib.request.urlopen(download_url)
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import webbrowser
import io
import os
from pathlib import Path
import glob
import pandas as pd
import time

In [ ]:
pattern = r"\([^()]*\)"
folder = 'C:/Users/Owner/Downloads'
file_list = []

# a bit of cleaning
dest_sample = 'PDFS/'
df = pd.read_csv('All.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

print(len(df_errors))


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


hash_list = []
for url in range(len(url_list)):
    time.sleep(2)
    if 'www' not in url_list[url]:
        print(url_list[url], url)
        driver.get(url_list[url])
        
        try:
            button = driver.find_element_by_css_selector('widget.buttonfieldwidget.xfaButton > input')
            print(button)
            button.click()
            df.at[index_list[url],'status'] = 0
            hash_list.append(hashme(url_list[url]))
        except:
            print('######################')
            print('ERROR')
            print('######################')
df.to_csv("All_cleaned2.csv",index=False, encoding="utf-8")



# for url in range(len(url_list)):
#     if 'download'==url_list[url].split('/')[-1]:
#         print(url_list[url], index_list[url])
#         status = download_pdf_unmasked(url_list[url],dest_sample,ids_list[url]+'.pdf')
#         if status == 'error':
#             print('#######################')
#             print('ERROR')
#             print('#######################')
#         else:
#             df.at[index_list[url],'status'] = 0

# df.to_csv("All_cleaned.csv",index=False, encoding="utf-8")


In [2]:
import hashlib
from selenium.webdriver.common.by import By
def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import webbrowser
import io
import os
from pathlib import Path
import glob
import pandas as pd
import time

In [ ]:

dest_sample = 'PDFS/'
df = pd.read_csv('All.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

#print(index_list)


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

hash_list = []
for url in range(76,len(url_list)):
    time.sleep(2)
    if 'www' not in url_list[url]:
        print(url_list[url], url)
        driver.get(url_list[url])
        
        driver.find_element_by_css_selector("[aria-label='Save as PDF'").click()#this is me most common layout. However, not nearly the only one..
        #as a result some where manually downloaded
        # df.at[index_list[url],'status'] = 0

#20, 31, 33, 37, 41

In [ ]:
folder = 'C:/Users/Owner/Downloads'
df = pd.read_csv('All_cleaned2.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

len(index_list)

In [4]:

folder = 'C:/Users/Owner/Downloads'
errors_set = set(list([20, 31, 33, 37, 41])) #these were unsuccesful attempts. 2 were docs and the other 4 needed to be filled out before downloading
successful_ids = []

for id in range(len(ids_list)):
    if id not in errors_set:
        successful_ids.append(ids_list[id])
    # else:
    #     print(ids_list[id])
# for i in errors_set:
#     print(ids_list[i])

# navigate in our downloads directory and rename the files we just downloaded
# os.chdir(folder)
# files = glob.glob("*.pdf") #but only the pdfs 
# files.sort(key=os.path.getmtime) # from the first (download) to the last so it is consistent with our database
# for id in range(len(successful_ids)):
   
#     filename = os.fsdecode(files[id])
#     print(filename)
#     dst = f"{successful_ids[id]}.pdf" 
#     src =f"{folder}/{filename}"
#     os.rename(src, dst)

In [6]:
df = pd.read_csv('All.csv')
df_errors = df.loc[df['status']==1]
url_list = list(df_errors['url'])
ids_list = list(df_errors['id'])
index_list = df_errors.index

# to_change = df[df['id']==successful_ids[6]]

for id in range(len(successful_ids)):
    to_change = df[df['id']==successful_ids[id]].index
    df.at[to_change,'status'] = 0
df.to_csv("Trial.csv",index=False, encoding="utf-8")

In [ ]:
retval = os.getcwd()
print(retval)
folder = 'PDFS/G'
os.chdir(folder)
files = glob.glob("*.pdf")
for file in files:
    print(file)
retval = os.getcwd()
os.chdir(retval)